In [177]:
import pandas as pd
import numpy as np

from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import f1_score

import seaborn as sns
import matplotlib.pyplot as plt

In [178]:
# Read model features data
modeling_features = pd.read_csv('../../data/model_features.csv')
all_features = pd.read_csv('../../data/transformed_data_men_21_to_24.csv')

modeling_features.head()

,won,underdog,underdog_opp,fan_favorite,fan_favorite_opp,de,oe,te,pace,physicality_score,sos,srs,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,orb,drb,trb,ast,stl,blk,pf,tov,tov%,poss,ast_per_poss,ast_per_fg,tov_per_poss,ast_to_tov,poss_per_game,game_win_rate,gpt_sent_score_avg,de_opp,oe_opp,te_opp,pace_opp,physicality_score_opp,sos_opp,srs_opp,fg_opp,fga_opp,fg%_opp,3p_opp,3pa_opp,3p%_opp,ft_opp,fta_opp,ft%_opp,orb_opp,drb_opp,trb_opp,ast_opp,stl_opp,blk_opp,pf_opp,tov_opp,tov%_opp,poss_opp,ast_per_poss_opp,ast_per_fg_opp,tov_per_poss_opp,ast_to_tov_opp,poss_per_game_opp,game_win_rate_opp,gpt_sent_score_avg_opp,conf_rank,conf_rank_opp,luck,luck_opp,choke_rate,choke_rate_opp,upset_rate,upset_rate_opp,3mean_plus_minus,3mean_plus_minus_opp,win_streak,win_streak_opp,matchup_win_rate
0,1,0,0,1,0,83.955939,105.172414,1.25271,72.0,9.405172,-6.37,6.27,27.103448,60.413793,0.449,7.62069,22.172414,0.344,13.896552,20.482759,0.678,11.344828,24.758621,36.103448,17.689655,9.724138,2.965517,19.103448,13.586207,16.2,39.074138,0.45272,0.652672,0.347703,1.30203,1.347384,0.827586,48.666667,98.254994,103.126160,1.049577,67.0,8.390000,0.30,1.65,24.800000,56.640000,0.438,8.040000,23.000000,0.350,11.800000,16.920000,0.697,10.120000,25.760000,35.880000,11.800000,7.360000,2.52,17.240000,13.560000,17.3,36.277000,0.325275,0.475806,0.373791,0.870206,1.451080,0.520000,73.333333,15.0,15.0,0.666667,0.000000,0.0,NaN,0.0,NaN,14.333333,-3.333333,0,0,NaN
1,1,0,0,0,1,83.955939,105.172414,1.25271,72.0,9.405172,-6.37,6.27,27.103448,60.413793,0.449,7.62069,22.172414,0.344,13.896552,20.482759,0.678,11.344828,24.758621,36.103448,17.689655,9.724138,2.965517,19.103448,13.586207,16.2,39.074138,0.45272,0.652672,0.347703,1.30203,1.347384,0.827586,48.666667,100.518548,106.741125,1.061905,69.0,9.027778,-8.22,-6.66,27.148148,59.888889,0.453,7.185185,21.222222,0.339,12.851852,17.111111,0.751,12.629630,21.851852,34.481481,13.222222,6.777778,2.00,17.481481,14.703704,17.8,37.350000,0.354009,0.487040,0.393673,0.899244,1.383333,0.518519,63.000000,15.0,22.0,0.666667,0.571429,0.0,NaN,0.0,NaN,14.333333,-0.333333,1,2,NaN
2,1,0,0,1,0,83.955939,105.172414,1.25271,72.0,9.405172,-6.37,6.27,27.103448,60.413793,0.449,7.62069,22.172414,0.344,13.896552,20.482759,0.678,11.344828,24.758621,36.103448,17.689655,9.724138,2.965517,19.103448,13.586207,16.2,39.074138,0.45272,0.652672,0.347703,1.30203,1.347384,0.827586,48.666667,108.277778,93.500000,0.863520,72.0,7.550000,-2.09,-12.73,25.000000,59.760000,0.418,5.320000,15.880000,0.335,12.000000,17.440000,0.688,8.480000,25.640000,34.120000,9.440000,5.640000,3.00,18.840000,13.080000,16.1,37.884000,0.249182,0.377600,0.345264,0.721713,1.515360,0.200000,59.333333,15.0,12.0,0.666667,0.444444,0.0,NaN,0.0,0.000000,14.333333,-5.666667,2,1,NaN
3,1,0,0,1,0,83.955939,105.172414,1.25271,72.0,9.405172,-6.37,6.27,27.103448,60.413793,0.449,7.62069,22.172414,0.344,13.896552,20.482759,0.678,11.344828,24.758621,36.103448,17.689655,9.724138,2.965517,19.103448,13.586207,16.2,39.074138,0.45272,0.652672,0.347703,1.30203,1.347384,0.827586,48.666667,90.289855,106.304348,1.177368,69.0,8.287500,-3.02,-7.28,26.500000,56.450000,0.469,5.500000,14.550000,0.378,14.850000,20.600000,0.721,8.500000,23.450000,31.950000,14.700000,10.450000,2.65,19.550000,11.550000,14.8,39.335000,0.373713,0.554717,0.293632,1.272727,1.966750,0.500000,68.666667,15.0,15.0,0.666667,0.000000,0.0,NaN,0.0,NaN,14.333333,-10.333333,3,0,0.5
4,0,1,0,0,1,83.955939,105.172414,1.25271,72.0,9.405172,-6.37,6.27,27.103448,60.413793,0.449,7.62069,22.172414,0.344,13.896552,20.482759,0.678,11.344828,24.758621,36.103448,17.689655,9.724138,2.965517,19.103448,13.586207,16.2,39.074138,0.45272,0.652672,0.347703,1.30203,1.347384,0.827586,48.666667,93.532338,107.309606,1.147300,67.0,8.353448,7.28,16.59,25.310345,57.655172,0.439,6.379310,18.034483,0.354,15.517241,21.862069,0.710,11.482759,23.482759,34.965517,13.000000,7.137931,4.00,17.310345,10.793103,13.7,35.005172,0.371374,0.513624,0.308329,1.204473,1.207075,0.620690,74.333333,15.0,1.0,0.66666

In [179]:
all_features.head()

,team,opponent,won,team_score,opponent_score,team_rank,opponent_rank,game_round,season_type,date,season_year,home_game,underdog,underdog_opp,fan_favorite,fan_favorite_opp,g,w,l,de,oe,te,pace,physicality_score,sos,srs,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,orb,drb,trb,ast,stl,blk,pf,tov,tov%,poss,ast_per_poss,ast_per_fg,tov_per_poss,ast_to_tov,poss_per_game,...,oe_opp,te_opp,pace_opp,physicality_score_opp,sos_opp,srs_opp,fg_opp,fga_opp,fg%_opp,3p_opp,3pa_opp,3p%_opp,ft_opp,fta_opp,ft%_opp,orb_opp,drb_opp,trb_opp,ast_opp,stl_opp,blk_opp,pf_opp,tov_opp,tov%_opp,poss_opp,ast_per_poss_opp,ast_per_fg_opp,tov_per_poss_opp,ast_to_tov_opp,poss_per_game_opp,game_win_rate_opp,gpt_sent_score_avg_opp,seed,conf,seed_opp,conf_opp,conf_rank,conf_rank_opp,plus_minus,luck,luck_opp,choke_rate,choke_rate_opp,upset_rate,upset_rate_opp,3mean_plus_minus,3mean_plus_minus_opp,win_streak,win_streak_opp,matchup_win_rate
0,Abilene Christian,East Tennessee State,1,70.0,47.0,NaN,NaN,NaN,regular_season,2020/11/25,2021,1,0,0,1,0,29.0,24.0,5.0,83.955939,105.172414,1.25271,72.0,9.405172,-6.37,6.27,27.103448,60.413793,0.449,7.62069,22.172414,0.344,13.896552,20.482759,0.678,11.344828,24.758621,36.103448,17.689655,9.724138,2.965517,19.103448,13.586207,16.2,39.074138,0.45272,0.652672,0.347703,1.30203,1.347384,...,103.126160,1.049577,67.0,8.390000,0.30,1.65,24.800000,56.640000,0.438,8.040000,23.000000,0.350,11.800000,16.920000,0.697,10.120000,25.760000,35.880000,11.800000,7.360000,2.52,17.240000,13.560000,17.3,36.277000,0.325275,0.475806,0.373791,0.870206,1.451080,0.520000,73.333333,14.0,Slnd,NaN,SC,15.0,15.0,23.0,0.666667,0.000000,0.0,NaN,0.0,NaN,14.333333,-3.333333,0,0,NaN
1,Abilene Christian,Austin Peay,1,80.0,72.0,NaN,NaN,NaN,regular_season,2020/11/27,2021,0,0,0,0,1,29.0,24.0,5.0,83.955939,105.172414,1.25271,72.0,9.405172,-6.37,6.27,27.103448,60.413793,0.449,7.62069,22.172414,0.344,13.896552,20.482759,0.678,11.344828,24.758621,36.103448,17.689655,9.724138,2.965517,19.103448,13.586207,16.2,39.074138,0.45272,0.652672,0.347703,1.30203,1.347384,...,106.741125,1.061905,69.0,9.027778,-8.22,-6.66,27.148148,59.888889,0.453,7.185185,21.222222,0.339,12.851852,17.111111,0.751,12.629630,21.851852,34.481481,13.222222,6.777778,2.00,17.481481,14.703704,17.8,37.350000,0.354009,0.487040,0.393673,0.899244,1.383333,0.518519,63.000000,14.0,Slnd,NaN,ASun,15.0,22.0,8.0,0.666667,0.571429,0.0,NaN,0.0,NaN,14.333333,-0.333333,1,2,NaN
2,Abilene Christian,Nebraska Omaha,1,70.0,58.0,NaN,NaN,NaN,regular_season,2020/11/28,2021,1,0,0,1,0,29.0,24.0,5.0,83.955939,105.172414,1.25271,72.0,9.405172,-6.37,6.27,27.103448,60.413793,0.449,7.62069,22.172414,0.344,13.896552,20.482759,0.678,11.344828,24.758621,36.103448,17.689655,9.724138,2.965517,19.103448,13.586207,16.2,39.074138,0.45272,0.652672,0.347703,1.30203,1.347384,...,93.500000,0.863520,72.0,7.550000,-2.09,-12.73,25.000000,59.760000,0.418,5.320000,15.880000,0.335,12.000000,17.440000,0.688,8.480000,25.640000,34.120000,9.440000,5.640000,3.00,18.840000,13.080000,16.1,37.884000,0.249182,0.377600,0.345264,0.721713,1.515360,0.200000,59.333333,14.0,Slnd,NaN,Sum,15.0,12.0,12.0,0.666667,0.444444,0.0,NaN,0.0,0.000000,14.333333,-5.666667,2,1,NaN
3,Abilene Christian,Tarleton State,1,69.0,48.0,NaN,NaN,NaN,regular_season,2020/12/05,2021,1,0,0,1,0,29.0,24.0,5.0,83.955939,105.172414,1.25271,72.0,9.405172,-6.37,6.27,27.103448,60.413793,0.449,7.62069,22.172414,0.344,13.896552,20.482759,0.678,11.344828,24.758621,36.103448,17.689655,9.724138,2.965517,19.103448,13.586207,16.2,39.074138,0.45272,0.652672,0.347703,1.30203,1.347384,...,106.304348,1.177368,69.0,8.287500,-3.02,-7.28,26.500000,56.450000,0.469,5.500000,14.550000,0.378,14.850000,20.600000,0.721,8.500000,23.450000,31.950000,14.700000,10.450000,2.65,19.550000,11.550000,14.8,39.335000,0.373713,0.554717,0.293632,1.272727,1.966750,0.500000,68.666667,14.0,Slnd,NaN,WAC,15.0,15.0,21.0,0.666667,0.000000,0.0,NaN,0.0,NaN,14.333333,-10.333333,3,0,NaN
4,Abilene Christian,Texas Tech,0,44.0,51.0,NaN,17.0,NaN,regular_season,2020

#### Remove all qualitative fields besides team names, team ranks, and season_year

In [180]:
exception_fields = ['team','seed','team_rank','opponent','seed_opp','opponent_rank','team_score','opponent_score','plus_minus',
                    'season_year','season_type','game_round']
qualitative_fields = [col for col in all_features.columns if (col not in modeling_features.columns) & (col not in exception_fields)]
qualitative_fields

['date',
 'home_game',
 'g',
 'w',
 'l',
 'g_opp',
 'w_opp',
 'l_opp',
 'conf',
 'conf_opp']

In [219]:
# Set baseline f1-score

df = all_features.loc[all_features.season_type == 'ncaa_tournament'].drop(qualitative_fields,axis=1)
# df = all_features.drop(qualitative_fields,axis=1)

# Split training and test data
X = df[df.columns[~df.columns.isin(['won'])]]
y = df['won']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.drop(exception_fields, axis=1, inplace=True)
# X_train_raw = X_train.copy() # To be concatenated later
# df_pred = X_test[exception_fields]
df_pred = X_test.copy()
X_test.drop(exception_fields, axis=1, inplace=True)

for scaler in [StandardScaler(), MinMaxScaler()]:
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

model = HistGradientBoostingClassifier(
    learning_rate = 0.05,
    min_samples_leaf = 25,
    random_state=42)

# Train the model on the training set
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

f1 = f1_score(y_test, y_pred)
print(f1)

0.6769230769230768


In [220]:
pd.options.display.max_columns = 100

In [221]:
df_pred['won'] = y_test
df_pred['prediction'] = y_pred
df_pred['correct_prediction'] = 1
df_pred.loc[df_pred.won != df_pred.prediction, 'correct_prediction'] = 0

pred = df_pred.reset_index(drop=True)#.join(X_train_raw.reset_index(drop=True))
pred['matchup_win_rate_opp'] = 1- pred.matchup_win_rate
failures = pred.loc[pred.correct_prediction == 0]
failures.loc[failures.season_type == 'ncaa_tournament'].sort_values('plus_minus').head()

,team,opponent,team_score,opponent_score,team_rank,opponent_rank,game_round,season_type,season_year,underdog,underdog_opp,fan_favorite,fan_favorite_opp,de,oe,te,pace,physicality_score,sos,srs,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,orb,drb,trb,ast,stl,blk,pf,tov,tov%,poss,ast_per_poss,ast_per_fg,tov_per_poss,ast_to_tov,poss_per_game,game_win_rate,gpt_sent_score_avg,de_opp,oe_opp,te_opp,pace_opp,physicality_score_opp,sos_opp,srs_opp,fg_opp,fga_opp,fg%_opp,3p_opp,3pa_opp,3p%_opp,ft_opp,fta_opp,ft%_opp,orb_opp,drb_opp,trb_opp,ast_opp,stl_opp,blk_opp,pf_opp,tov_opp,tov%_opp,poss_opp,ast_per_poss_opp,ast_per_fg_opp,tov_per_poss_opp,ast_to_tov_opp,poss_per_game_opp,game_win_rate_opp,gpt_sent_score_avg_opp,seed,seed_opp,conf_rank,conf_rank_opp,plus_minus,luck,luck_opp,choke_rate,choke_rate_opp,upset_rate,upset_rate_opp,3mean_plus_minus,3mean_plus_minus_opp,win_streak,win_streak_opp,matchup_win_rate,won,prediction,correct_prediction,matchup_win_rate_opp
34,North Carolina,Wisconsin,62.0,85.0,8.0,9.0,FIRST ROUND,ncaa_tournament,2021,0,1,0,1,98.494415,105.925206,1.075444,71.0,10.500000,9.52,14.79,27.620690,62.965517,0.439,5.586207,17.551724,0.318,14.379310,21.517241,0.668,15.655172,27.172414,42.827586,15.275862,7.034483,4.724138,16.758621,14.586207,16.6,36.772414,0.415416,0.553059,0.396662,1.047281,1.268014,0.620690,81.333333,98.609231,106.683077,1.081877,65.0,6.717742,11.93,17.22,24.741935,58.258065,0.425,8.677419,23.806452,0.364,11.741935,15.290323,0.768,8.225806,25.387097,33.612903,13.354839,5.645161,3.967742,16.129032,9.032258,12.1,32.811290,0.407020,0.539765,0.275279,1.478571,1.058429,0.580645,71.666667,8.0,9.0,2.0,4.0,-23.0,0.500000,0.250000,0.285714,0.200000,0.333333,0.100,15.666667,-2.666667,0,0,NaN,0,1,0,NaN
23,Pittsburgh,Xavier,73.0,84.0,11.0,3.0,SECOND ROUND,ncaa_tournament,2023,1,0,1,0,102.532680,110.457516,1.077291,68.0,7.611111,5.20,10.59,25.833333,57.805556,0.447,9.027778,25.083333,0.360,14.416667,19.000000,0.759,10.111111,26.027778,36.138889,14.055556,5.138889,4.083333,16.250000,11.111111,14.3,35.858333,0.391975,0.544086,0.309861,1.265000,0.996065,0.666667,79.333333,102.543958,112.046390,1.092667,72.0,8.074324,9.16,16.03,29.972973,60.972973,0.492,7.405405,19.000000,0.390,13.594595,19.135135,0.710,10.270270,27.459459,37.729730,19.054054,6.378378,3.243243,16.027027,12.405405,15.0,41.197297,0.462507,0.635708,0.301122,1.535948,1.113440,0.729730,74.000000,11.0,3.0,2.0,2.0,-11.0,0.636364,0.500000,1.000000,0.222222,0.555556,0.375,-2.666667,4.333333,2,1,NaN,0,1,0,NaN
72,Alabama,UCLA,78.0,88.0,2.0,11.0,SWEET 16,ncaa_tournament,2021,0,1,0,1,94.360020,107.251402,1.136619,74.0,9.765152,10.01,19.58,28.000000,64.393939,0.435,10.545455,29.969697,0.352,13.121212,18.545455,0.708,12.363636,28.333333,40.696970,14.060606,8.484848,4.333333,18.272727,13.878788,15.9,38.324242,0.366885,0.502165,0.362141,1.013100,1.161341,0.787879,85.000000,101.712627,109.097242,1.072603,65.0,6.992188,11.34,16.30,26.750000,57.968750,0.461,6.812500,18.312500,0.372,13.093750,18.312500,0.715,9.750000,25.156250,34.906250,13.656250,4.937500,2.781250,16.062500,10.500000,13.6,36.198437,0.377261,0.510514,0.290068,1.300595,1.131201,0.687500,80.000000,2.0,11.0,5.0,6.0,-10.0,0.714286,0.545455,0.200000,0.444444,0.750000,0.600,11.000000,12.333333,7,3,NaN,0,1,0,NaN
10,Alabama,San Diego State,64.0,71.0,1.0,5.0,SWEET 16,ncaa_tournament,2023,0,1,0,1,92.935982,111.368653,1.198337,72.0,9.520270,9.65,23.19,27.648649,62.540541,0.442,9.891892,29.540541,0.335,16.621622,22.918919,0.725,13.081081,31.567568,44.648649,15.000000,6.054054,5.108108,18.675676,13.837838,15.9,39.291892,0.381758,0.542522,0.352180,1.083984,1.061943,0.837838,85.000000,95.541401,107.122177,1.121212,66.0,8.275641,8.92,15.81,25.282051,57.717949,0.438,6.948718,19.948718,0.348,13.641026,18.897436,0.722,10.923077,25.384615,36.307692,12.923077,6.717949,3.871795,16.256410,11.589744,14.8,34.925000,0.370024,0.511156,0.331847,1.115044,0.895513,0.820513,89.000000,1.0,5.0,5.0,9.0,-7.0,1.000000,0.875000,0.161290,0.150000,0.833333,0.625

In [222]:
failures_to_examine = [349, 4200, 1788]
df_f = failures.loc[failures_to_examine].set_index(['team','opponent','seed','seed_opp','season_year','game_round','prediction','won','plus_minus'])

KeyError: "None of [Index([349, 4200, 1788], dtype='int32')] are in the [index]"

In [223]:
fields_to_compare = ['tov_per_poss','game_win_rate','fan_favorite','ast_per_poss','sos','te']
fields_to_compare_complete = sorted(fields_to_compare+[col + '_opp' for col in fields_to_compare])

In [224]:
all_features.loc[(all_features.team == 'Houston') & (all_features.opponent == 'Baylor')]

,team,opponent,won,team_score,opponent_score,team_rank,opponent_rank,game_round,season_type,date,season_year,home_game,underdog,underdog_opp,fan_favorite,fan_favorite_opp,g,w,l,de,oe,te,pace,physicality_score,sos,srs,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,orb,drb,trb,ast,stl,blk,pf,tov,tov%,poss,ast_per_poss,ast_per_fg,tov_per_poss,ast_to_tov,poss_per_game,...,oe_opp,te_opp,pace_opp,physicality_score_opp,sos_opp,srs_opp,fg_opp,fga_opp,fg%_opp,3p_opp,3pa_opp,3p%_opp,ft_opp,fta_opp,ft%_opp,orb_opp,drb_opp,trb_opp,ast_opp,stl_opp,blk_opp,pf_opp,tov_opp,tov%_opp,poss_opp,ast_per_poss_opp,ast_per_fg_opp,tov_per_poss_opp,ast_to_tov_opp,poss_per_game_opp,game_win_rate_opp,gpt_sent_score_avg_opp,seed,conf,seed_opp,conf_opp,conf_rank,conf_rank_opp,plus_minus,luck,luck_opp,choke_rate,choke_rate_opp,upset_rate,upset_rate_opp,3mean_plus_minus,3mean_plus_minus_opp,win_streak,win_streak_opp,matchup_win_rate
12450,Houston,Baylor,0,59.0,78.0,2.0,1.0,FINAL FOUR,ncaa_tournament,2021/04/03,2021,0,1,0,1,0,32.0,28.0,4.0,88.210227,115.246212,1.306495,66.0,9.398438,5.37,21.66,26.687500,61.28125,0.435,9.093750,25.718750,0.354,13.593750,18.593750,0.731,14.312500,26.1875,40.500000,13.937500,7.968750,4.656250,17.968750,10.656250,13.2,31.863281,0.437416,0.522248,0.334437,1.307918,0.995728,...,119.646401,1.266293,69.0,9.141667,7.40,24.83,30.566667,62.866667,0.486,9.966667,24.133333,0.413,11.800000,16.666667,0.708,12.466667,23.500000,35.966667,16.900000,8.900000,3.700000,17.100000,11.500000,14.0,37.516667,0.450466,0.552890,0.306530,1.469565,1.250556,0.933333,82.0,2.0,Amer,1.0,B12,8.0,1.0,-19.0,0.666667,NaN,0.125000,0.083333,0.833333,1.00,8.333333,11.000000,10,4,NaN
12552,Houston,Baylor,1,82.0,76.0,2.0,11.0,NaN,regular_season,2024/02/24,2024,0,0,1,0,1,28.0,25.0,3.0,87.057522,114.491150,1.315121,64.0,9.562500,8.99,26.71,26.571429,61.25000,0.434,7.464286,21.821429,0.342,13.321429,19.357143,0.688,14.285714,24.5000,38.785714,13.071429,10.428571,4.821429,17.964286,8.714286,11.0,30.194643,0.432906,0.491935,0.288604,1.500000,1.078380,...,120.421607,1.153458,66.0,8.455357,10.31,20.60,28.107143,57.714286,0.487,8.857143,22.321429,0.397,16.535714,22.571429,0.733,11.535714,23.678571,35.214286,15.321429,6.892857,3.321429,16.785714,12.071429,15.0,39.364286,0.389222,0.545108,0.306659,1.269231,1.405867,0.714286,82.0,NaN,B12,NaN,B12,1.0,1.0,6.0,0.500000,0.428571,0.111111,0.238095,NaN,0.25,11.333333,7.666667,4,0,0.0


In [225]:
df_f[fields_to_compare_complete].T

team,Saint Mary's (CA),Arkansas,San Diego State
opponent,UCLA,Kansas,Connecticut
seed,5.0,8.0,5.0
seed_opp,4.0,1.0,4.0
season_year,2022,2023,2023
game_round,SECOND ROUND,SECOND ROUND,CHAMPIONSHIP
prediction,1,0,1
won,0,1,0
plus_minus,-16.0,1.0,-17.0
ast_per_poss,0.383256,0.322908,0.370024
ast_per_poss_opp,0.409712,0.422477,0.481851


1. From a seeding perspective, the only team that should possibly have a fan_favorite underdog score would be Arkansas vs Kansas. The simple seeding/ranking calculation could be superficial as San Diego State deserved an underdog rating vs UConn by every means.
2. Tournament data vs Regular Season data
3. Matchup Winrate's calculation may be superficial - win recency and/or frequency should be factored into the score somehow

In [226]:
pred[['season_type','correct_prediction']].groupby(['season_type']).sum() / pred[['season_type','correct_prediction']].groupby(['season_type']).count()

,correct_prediction
season_type,
ncaa_tournament,0.7375
